In [1]:
### import libraries
import torch
import torch.nn as nn
import numpy as np

# Explore the LSTM type

In [2]:
# set layer parameters
input_size  =  9 # number of features to extract (e.g., number of data channels)
hidden_size = 16 # number of units in the hidden state
num_layers  =  2 # number of vertical stacks of hidden layers (note: only the final layer gives an output)

# create an LSTM instance
lstm = nn.LSTM(input_size, hidden_size, num_layers)
lstm

LSTM(9, 16, num_layers=2)

In [3]:
# check out the source code for more detailed info about this class
??nn.LSTM

Init signature: nn.LSTM(*args, **kwargs)
Source:        
class LSTM(RNNBase):
    r"""Applies a multi-layer long short-term memory (LSTM) RNN to an input
    sequence.


    For each element in the input sequence, each layer computes the following
    function:

    .. math::
        \begin{array}{ll} \\
            i_t = \sigma(W_{ii} x_t + b_{ii} + W_{hi} h_{t-1} + b_{hi}) \\
            f_t = \sigma(W_{if} x_t + b_{if} + W_{hf} h_{t-1} + b_{hf}) \\
            g_t = \tanh(W_{ig} x_t + b_{ig} + W_{hg} h_{t-1} + b_{hg}) \\
            o_t = \sigma(W_{io} x_t + b_{io} + W_{ho} h_{t-1} + b_{ho}) \\
            c_t = f_t \odot c_{t-1} + i_t \odot g_t \\
            h_t = o_t \odot \tanh(c_t) \\
        \end{array}

    where :math:`h_t` is the hidden state at time `t`, :math:`c_t` is the cell
    state at time `t`, :math:`x_t` is the input at time `t`, :math:`h_{t-1}`
    is the hidden state of the layer at time `t-1` or the initial hidden
    state at time `0`, and :math:`i_t`, :math:`f

In [4]:
# set data parameters
seqlength = 5
batchsize = 2

# create some data
X = torch.rand(seqlength,batchsize,input_size)

# create initial hidden states (typically initialized as zeros)
H = torch.zeros(num_layers,batchsize,hidden_size)
C = torch.zeros(num_layers,batchsize,hidden_size)

# the input is actually a tuple of (hidden,cell)
hiddeninputs = (H, C)

# run some data through the model and show the output sizes
y,h = lstm(X,hiddeninputs)
print(f' Input shape: {list(X.shape)}')
print(f'Hidden shape: {list(h[0].shape)}')
print(f'  Cell shape: {list(h[1].shape)}')
print(f'Output shape: {list(y.shape)}')

 Input shape: [5, 2, 9]
Hidden shape: [2, 2, 16]
  Cell shape: [2, 2, 16]
Output shape: [5, 2, 16]


In [5]:
# Check out the learned parameters and their sizes
for p in lstm.named_parameters():
  if 'weight' in p[0]:
    print(f'{p[0]} has size {list(p[1].shape)}')

weight_ih_l0 has size [64, 9]
weight_hh_l0 has size [64, 16]
weight_ih_l1 has size [64, 16]
weight_hh_l1 has size [64, 16]


# Create a DL model class

In [6]:
class LSTMnet(nn.Module):
  def __init__(self,input_size,num_hidden,num_layers):
    super().__init__()

    # store parameters
    self.input_size = input_size
    self.num_hidden = num_hidden
    self.num_layers = num_layers

    # RNN Layer (notation: LSTM \in RNN)
    self.lstm = nn.LSTM(input_size,num_hidden,num_layers)
    
    # linear layer for output
    self.out = nn.Linear(num_hidden, 1)
  
  def forward(self,x):
    
    print(f'Input: {list(x.shape)}')

    # run through the RNN layer
    y,hidden = self.lstm(x)
    print(f'RNN-out: {list(y.shape)}')
    print(f'RNN-hidden: {list(hidden[0].shape)}')
    print(f'RNN-cell: {list(hidden[1].shape)}')
    
    # pass the RNN output through the linear output layer
    o = self.out(y)
    print(f'Output: {list(o.shape)}')

    return o,hidden

In [7]:
# create an instance of the model and inspect
net = LSTMnet(input_size,hidden_size,num_layers)
print(net), print(' ')

# and check out all learnable parameters
for p in net.named_parameters():
  print(f'{p[0]:>20} has size {list(p[1].shape)}')

LSTMnet(
  (lstm): LSTM(9, 16, num_layers=2)
  (out): Linear(in_features=16, out_features=1, bias=True)
)
 
   lstm.weight_ih_l0 has size [64, 9]
   lstm.weight_hh_l0 has size [64, 16]
     lstm.bias_ih_l0 has size [64]
     lstm.bias_hh_l0 has size [64]
   lstm.weight_ih_l1 has size [64, 16]
   lstm.weight_hh_l1 has size [64, 16]
     lstm.bias_ih_l1 has size [64]
     lstm.bias_hh_l1 has size [64]
          out.weight has size [1, 16]
            out.bias has size [1]


In [8]:
# test the model with some data
# create some data
X = torch.rand(seqlength,batchsize,input_size)
y = torch.rand(seqlength,batchsize,1)
yHat,h = net(X)


lossfun = nn.MSELoss()
lossfun(yHat,y)

Input: [5, 2, 9]
RNN-out: [5, 2, 16]
RNN-hidden: [2, 2, 16]
RNN-cell: [2, 2, 16]
Output: [5, 2, 1]


tensor(0.3819, grad_fn=<MseLossBackward0>)

# GRU

In [9]:
# create a GRU instance
gru = nn.GRU(input_size, hidden_size, num_layers)
gru

GRU(9, 16, num_layers=2)

In [10]:
??nn.GRU

Init signature: nn.GRU(*args, **kwargs)
Source:        
class GRU(RNNBase):
    r"""Applies a multi-layer gated recurrent unit (GRU) RNN to an input sequence.


    For each element in the input sequence, each layer computes the following
    function:

    .. math::
        \begin{array}{ll}
            r_t = \sigma(W_{ir} x_t + b_{ir} + W_{hr} h_{(t-1)} + b_{hr}) \\
            z_t = \sigma(W_{iz} x_t + b_{iz} + W_{hz} h_{(t-1)} + b_{hz}) \\
            n_t = \tanh(W_{in} x_t + b_{in} + r_t * (W_{hn} h_{(t-1)}+ b_{hn})) \\
            h_t = (1 - z_t) * n_t + z_t * h_{(t-1)}
        \end{array}

    where :math:`h_t` is the hidden state at time `t`, :math:`x_t` is the input
    at time `t`, :math:`h_{(t-1)}` is the hidden state of the layer
    at time `t-1` or the initial hidden state at time `0`, and :math:`r_t`,
    :math:`z_t`, :math:`n_t` are the reset, update, and new gates, respectively.
    :math:`\sigma` is the sigmoid function, and :math:`*` is the Hadamard product.

    In 

In [11]:
# create some data and a hidden state
X = torch.rand(seqlength,batchsize,input_size)
H = torch.zeros(num_layers,batchsize,hidden_size)

# run some data through the model and show the output sizes
y,h = gru(X,H) # No cell states in GRU!
print(f' Input shape: {list(X.shape)}')
print(f'Hidden shape: {list(h.shape)}')
print(f'Output shape: {list(y.shape)}')

 Input shape: [5, 2, 9]
Hidden shape: [2, 2, 16]
Output shape: [5, 2, 16]


In [12]:
# Check out the learned parameters and their sizes
for p in gru.named_parameters():
  print(f'{p[0]:>15} has size {list(p[1].shape)}')

   weight_ih_l0 has size [48, 9]
   weight_hh_l0 has size [48, 16]
     bias_ih_l0 has size [48]
     bias_hh_l0 has size [48]
   weight_ih_l1 has size [48, 16]
   weight_hh_l1 has size [48, 16]
     bias_ih_l1 has size [48]
     bias_hh_l1 has size [48]
